# Taller 06: Base de Datos Vectoriales

### Parte 1: Recuperación con TF-IDF

**1. Carga los datos en Python**

In [2]:
import pandas as pd

# Carga el archivo CSV en un DataFrame
df = pd.read_csv("../data/wiki_movie_plots_deduped.csv")

In [ ]:
# Filtra únicamente las columnas Title y Plot
df_filtered = df[['Title', 'Plot']]

# Muestra las primeras filas del DataFrame filtrado
print(df_filtered.head())

In [3]:
# Filtra únicamente las columnas Title y Plot
df_filtered = df[['Title', 'Plot']]

# Muestra las primeras filas del DataFrame filtrado
print(df_filtered.head())

                              Title  \
0            Kansas Saloon Smashers   
1     Love by the Light of the Moon   
2           The Martyred Presidents   
3  Terrible Teddy, the Grizzly King   
4            Jack and the Beanstalk   

                                                Plot  
0  A bartender is working at a saloon, serving dr...  
1  The moon, painted with a smiling face hangs ov...  
2  The film, just over a minute long, is composed...  
3  Lasting just 61 seconds and consisting of two ...  
4  The earliest known adaptation of the classic f...  


**2. Configurar TF-IDF**

- usa la libreria scikit-lear para calcular los puntajes TF-IDF de los plots

In [4]:
import unicodedata
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Función para limpiar texto
def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar tildes
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'
    )
    # Eliminar números, puntuaciones y caracteres no alfabéticos
    text = re.sub(r'[^a-z\s]', '', text)
    return text

# Inicializa el vectorizador TF-IDF con el preprocesador personalizado
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,  # Limita el vocabulario
    preprocessor=clean_text,  # Aplica la limpieza personalizada
    token_pattern=r'\b[a-z]{2,}\b'  # Solo considera palabras de al menos 2 letras
)

# Calcula los puntajes TF-IDF para los Plots
tfidf_matrix = tfidf_vectorizer.fit_transform(df_filtered['Plot'].fillna(''))

# Convierte la matriz dispersa en un DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=df_filtered['Title']
)

# Muestra las primeras filas del DataFrame TF-IDF
print(tfidf_df.head())

                                  aaron  abandon  abandoned  abandons  abby  \
Title                                                                         
Kansas Saloon Smashers              0.0      0.0        0.0       0.0   0.0   
Love by the Light of the Moon       0.0      0.0        0.0       0.0   0.0   
The Martyred Presidents             0.0      0.0        0.0       0.0   0.0   
Terrible Teddy, the Grizzly King    0.0      0.0        0.0       0.0   0.0   
Jack and the Beanstalk              0.0      0.0        0.0       0.0   0.0   

                                  abducted  abhi  abilities  ability  \
Title                                                                  
Kansas Saloon Smashers                 0.0   0.0        0.0      0.0   
Love by the Light of the Moon          0.0   0.0        0.0      0.0   
The Martyred Presidents                0.0   0.0        0.0      0.0   
Terrible Teddy, the Grizzly King       0.0   0.0        0.0      0.0   
Jack and the B

**3. Realizar Consultas:**

- Escribe una función que calculo la similitud entre una consulta y los documentos usando la matriz TF-IDF

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

def calcular_similitud(query, tfidf_vectorizer, tfidf_matrix, titles):
    # Limpia la consulta usando la misma lógica de preprocesamiento
    query_cleaned = clean_text(query)

    # Vectoriza la consulta
    query_tfidf = tfidf_vectorizer.transform([query_cleaned])

    # Calcula la similitud coseno entre la consulta y los documentos
    similitudes = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    # Crea un DataFrame con los resultados
    resultados = pd.DataFrame({
        'Title': titles,
        'Similarity': similitudes
    })

    # Ordena los documentos por similitud en orden descendente
    resultados_ordenados = resultados.sort_values(by='Similarity', ascending=False)

    return resultados_ordenados

# Ejemplo de uso
consulta = "man"
resultados = calcular_similitud(consulta, tfidf_vectorizer, tfidf_matrix, df_filtered['Title'])

# Muestra los resultados
print(resultados.head(10))

                                 Title  Similarity
18255                 The Medicine Man    0.452338
18965                Death Is a Number    0.440332
20811       My Wrongs #8245–8249 & 117    0.392925
22406                        King Dave    0.335430
18390            The Man in the Mirror    0.331631
1942                  Murder in Harlem    0.330803
15734                         The Road    0.324203
16472  Cheech & Chong's Animated Movie    0.316461
23866              Everyday I Love You    0.314217
27894                    Junior Senior    0.307387


**4. Evaluar los resultados:**

- Registra los documentos recuperados y analiza su relevancia

In [7]:
import os

def registrar_documentos(resultados, output_path="resultados_recuperados.csv", threshold=0.5):
    # Asegúrate de que el directorio existe
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Marca los documentos como relevantes o no según el umbral
    resultados['Relevancia'] = resultados['Similarity'] >= threshold

    # Guarda los resultados en un archivo CSV
    resultados.to_csv(output_path, index=False)
    print(f"Resultados guardados en: {output_path}")

    # Filtra los documentos relevantes
    resultados_filtrados = resultados[resultados['Relevancia']]
    print(f"Documentos relevantes (similitud >= {threshold}): {len(resultados_filtrados)}")

    return resultados_filtrados

# Llama a la función con los resultados de la consulta
ruta_salida = os.path.join("data", "resultados_recuperados.csv")
documentos_relevantes = registrar_documentos(resultados, output_path=ruta_salida, threshold=0.4)

# Analiza los resultados relevantes
print("Documentos relevantes:")
print(documentos_relevantes[['Title', 'Similarity']].head(10))

Resultados guardados en: data\resultados_recuperados.csv
Documentos relevantes (similitud >= 0.4): 2
Documentos relevantes:
                   Title  Similarity
18255   The Medicine Man    0.452338
18965  Death Is a Number    0.440332


### Parte 2: Recuperación con BM25

**1. Configurar Elasticsearch:**

- Reutiliza el índice creado en el Ejercicio 1 para realizar consultas basadas en BM25

In [8]:
pip install elasticsearch


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import pandas as pd

# Conectar a Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Cargar datos del archivo CSV
df = pd.read_csv("../data/wiki_movie_plots_deduped.csv")
df_filtered = df[['Title', 'Plot']]

# Crear un índice en Elasticsearch
index_name = "movies_bm25"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Title": {"type": "text"},
            "Plot": {"type": "text"}
        }
    }
})

# Insertar los documentos en el índice
def generate_data(df):
    for _, row in df.iterrows():
        yield {
            "_index": index_name,
            "_source": {
                "Title": row['Title'],
                "Plot": row['Plot']
            }
        }

bulk(es, generate_data(df_filtered))
print(f"Índice '{index_name}' creado y documentos insertados.")

**2. Realizar consultas usando BM25**

- Configura y ejecuta consultas: Realiza consultas a Elasticsearch con las palabras clave, como "dinosaurs" o "cyborg".


In [ ]:
# Función para realizar consultas
def search_query(query, index_name):
    response = es.search(
        index=index_name,
        body={
            "query": {
                "match": {
                    "Plot": query
                }
            }
        }
    )
    return response

# Consultas de ejemplo
keywords = ["dinosaurs", "cyborg"]
for keyword in keywords:
    results = search_query(keyword, index_name)
    print(f"Resultados para la consulta '{keyword}':")
    for hit in results['hits']['hits']:
        print(f"Title: {hit['_source']['Title']}, Score: {hit['_score']}")

**3. Evaluar los resultados**

- Compara con los resultados de TF-IDF:


In [ ]:
import pandas as pd

# Recuperar títulos y similitudes para BM25
def get_bm25_results(query, index_name):
    results = search_query(query, index_name)
    return pd.DataFrame([
        {"Title": hit["_source"]["Title"], "BM25_Score": hit["_score"]}
        for hit in results["hits"]["hits"]
    ])

# Comparar TF-IDF y BM25
consulta = "dinosaurs"
bm25_results = get_bm25_results(consulta, index_name)
tfidf_results = calcular_similitud(consulta, tfidf_vectorizer, tfidf_matrix, df_filtered['Title'])

# Merge y comparación
comparison = pd.merge(
    bm25_results, 
    tfidf_results.rename(columns={"Similarity": "TF-IDF_Score"}), 
    on="Title", 
    how="outer"
).fillna(0)

# Exportar a CSV para análisis
comparison.to_csv("comparison_results.csv", index=False)
print("Comparación exportada a 'comparison_results.csv'.")